In [1]:
import os
from pathlib import Path
import sys

# !IMPORTANT!, else Hydra isnt working !
current_path = os.getcwd()
sys.path.insert(0, Path(current_path).absolute().parents[0].as_posix())

import cv2
import tensorflow_datasets as tfds
import tensorflow as tf
import tqdm
import mediapy
import numpy as np
import torch
from uha import make_pytorch_oxe_iterable_dataset, get_octo_dataset_tensorflow, get_single_dataset_tensorflow
from torchvision.transforms.functional import convert_image_dtype
import omegaconf
from omegaconf import OmegaConf

/home/marcelr/miniforge3/envs/medit_policy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-08 08:12:42.775033: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-08 08:12:42.775103: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-08 08:12:42.776700: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-08 08:12:42.785826: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Te

## Load Data
Next, we will load a trajectory from the Bridge dataset. We will use the one defined in uha/data/oxe/oxe_dataset_configs.py.

In [2]:
cfg = omegaconf.OmegaConf.load("../uha/data/conf/uha_default_load_config.yaml")
del cfg.interleaved_dataset_cfg.frame_transform_kwargs.image_augment_kwargs # comment out to get augmented data
cfg_transforms = omegaconf.OmegaConf.load("../uha/data/conf/transforms/oxe_no_remapping.yaml")

In [3]:
# dataset = get_octo_dataset_tensorflow(cfg, train=True)
dataset = get_single_dataset_tensorflow(cfg, train=True)
is_single_dataset = True
batch_size = 128
# create Pytorch Train Dataset
dataloader = make_pytorch_oxe_iterable_dataset(dataset, train=True, batch_size=batch_size, transform_dict=cfg_transforms, num_workers=0, pin_memory=True, is_single_dataset=is_single_dataset, main_process=False)
# print("len in colab", len(dataloader.dataset))
it = iter(dataloader)
batch = next(it)
batch = next(it)
batch = next(it)
images, image_secondary, image_wrist = [], [], []
for i in range(batch_size):
  images.append(batch["observation"]["image_primary"][i, 0].numpy()) # [batch_size, window_size, rgb, width, height]
  image_secondary.append(batch["observation"]["image_secondary"][i, 0].numpy()) # [batch_size, window_size, rgb, width, height]
  # image_wrist.append(batch["observation"]["image_wrist"][i, 0].numpy()) # [batch_size, window_size, rgb, width, height]
  print("action", batch["action"][i, :, 0, :])

mediapy.show_video(images, fps=10)
mediapy.show_video(image_secondary, fps=10)
# mediapy.show_video(image_wrist, fps=10)

########################################
constructing single val dataset: libero_spatial_no_noops
########################################


2024-10-08 08:12:47.562948: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization


Cause: Unable to locate the source code of <function _gcd_import at 0x7f16ce34b400>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function _gcd_import at 0x7f16ce34b400>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function _gcd_import at 0x7f16ce34b400>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2024-10-08 08:12:48.680869: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-10-08 08:12:58.161984: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 66081259520 bytes after encountering the first element of size 32266240 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
/home/marcelr/miniforge3/envs/medit_policy/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py:223: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.c

action tensor([[-0.1141,  0.9582, -0.1977,  0.0152, -0.1453, -0.0390,  0.0000]])
action tensor([[-0.1141,  0.9826, -0.3238,  0.0152, -0.1508, -0.0584,  0.0000]])
action tensor([[-0.1141,  0.9965, -0.4327, -0.0254, -0.1732, -0.0455,  0.0000]])
action tensor([[-0.1141,  0.9547, -0.4871, -0.0355, -0.1341, -0.0260,  0.0000]])
action tensor([[-0.1141,  0.7422, -0.6074,  0.0152,  0.0782,  0.0065,  0.0000]])
action tensor([[-0.1364,  0.5575, -0.6819,  0.0152,  0.0782,  0.0455,  0.0000]])
action tensor([[-0.2128,  0.2962, -0.7994,  0.0152,  0.1676,  0.0455,  0.0000]])
action tensor([[-0.3592,  0.0035, -0.8797,  0.1371,  0.3073,  0.0649,  0.0000]])
action tensor([[-0.4292, -0.0732, -0.8911,  0.1574,  0.2682,  0.0649,  0.0000]])
action tensor([[-0.4961, -0.1045, -0.8911,  0.0761,  0.2570,  0.0714,  0.0000]])
action tensor([[-0.4897, -0.0906, -0.8797,  0.0863,  0.2235,  0.0779,  0.0000]])
action tensor([[-0.4451, -0.1150, -0.8682,  0.1168,  0.2458,  0.0714,  0.0000]])
action tensor([[-0.3624, -0.

This browser does not support the video tag.
